## 第10章 数据聚合与分组运算

pandas提供了⼀个灵活⾼效的gruopby功能，它使你能以⼀种⾃
然的⽅式对数据集进⾏切⽚、切块、摘要等操作

- 计算分组摘要统计，如计数、平均值、标准差，或⽤户⾃定义函数。
- 计算分组的概述统计，⽐如数量、平均值或标准差，或是⽤户定义的函数。
- 应⽤组内转换或其他运算，如规格化、线性回归、排名或选取⼦集等。
- 计算透视表或交叉表。
- 执⾏分位数分析以及其它统计分组分析。

In [1]:
import numpy as np
import pandas as pd

### 10.1 GroupBy机制

Hadley Wickham（许多热⻔R语⾔包的作者）创造了⼀个⽤于表示分组运算的术语"split-apply-combine"（拆分－应⽤－合并）。

第⼀个阶段， pandas对象（⽆论是Series、 DataFrame还是其他的）中的数据会根据你所提供的⼀个或多个键被拆分(split）为多组。拆分操作是在对象的特定轴上执⾏的。例如，DataFrame可以在其⾏（axis=0）或列axis=1）上进⾏分组。


然后，将⼀个函数应⽤（apply）到各个分组并产⽣⼀个新值。

最后，所有这些函数的执⾏结果会被合并（combine）到最终的结果对象中。结果对象的形式⼀般取决于数据上所执⾏的操作。



分组键可以有多种形式，且类型不必相同：
- 列表或数组，其⻓度与待分组的轴⼀样。
- 表示DataFrame某个列名的值。
- 字典或Series，给出待分组轴上的值与分组名之间的对应关系。
- 函数，⽤于处理轴索引或索引中的各个标签。

注意，后三种都只是快捷⽅式⽽已，其最终⽬的仍然是产⽣⼀组
⽤于拆分对象的值。如果觉得这些东⻄看起来很抽象，不⽤担
⼼，我将在本章中给出⼤量有关于此的示例。⾸先来看看下⾯这
个⾮常简单的表格型数据集（以DataFrame的形式）：

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.537970,-1.227607
1,a,two,0.324963,-0.852639
2,b,one,-1.623929,-0.204281
3,b,two,0.332654,-0.241716
4,a,one,0.620406,-1.349194


假设你想要按key1进⾏分组，并计算data1列的平均值。实现该
功能的⽅式有很多，⽽我们这⾥要⽤的是：访问data1，并根据
key1调⽤groupby

In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

变量grouped是⼀个GroupBy对象。它实际上还没有进⾏任何计
算，只是含有⼀些有关分组键df['key1']的中间数据⽽已。换句话
说，该对象已经有了接下来对各分组执⾏运算所需的⼀切信息。
例如，我们可以调⽤GroupBy的mean⽅法来计算分组平均值

In [5]:
grouped.mean()

key1
a    0.827779
b   -0.645638
Name: data1, dtype: float64

稍后我将详细讲解.mean()的调⽤过程。这⾥最重要的是，数据
（Series）根据分组键进⾏了聚合，产⽣了⼀个新的Series，其
索引为key1列中的唯⼀值。之所以结果中索引的名称为key1，是
因为原始DataFrame的列df['key1']就叫这个名字。

如果我们⼀次传⼊多个数组的列表，就会得到不同的结果：

In [10]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()  # 注意这里是两层括号
means

key1  key2
a     one     1.079188
      two     0.324963
b     one    -1.623929
      two     0.332654
Name: data1, dtype: float64

这⾥，我通过两个键对数据进⾏了分组，得到的Series具有⼀个
层次化索引（由唯⼀的键对组成）：

In [12]:
means.unstack()

key2,one,two
key1,,
a,1.079188,0.324963
b,-1.623929,0.332654


在这个例⼦中，分组键均为Series。实际上，分组键可以是任何
⻓度适当的数组：

In [14]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.324963
            2006   -1.623929
Ohio        2005    0.935312
            2006    0.620406
Name: data1, dtype: float64

通常，分组信息就位于相同的要处理DataFrame中。这⾥，你还
可以将列名（可以是字符串、数字或其他Python对象）⽤作分组
键

In [15]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.827779,-1.143147
b,-0.645638,-0.222998


In [16]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.079188 -1.288401
     two   0.324963 -0.852639
b    one  -1.623929 -0.204281
     two   0.332654 -0.241716

你可能已经注意到了，第⼀个例⼦在执⾏
df.groupby('key1').mean()时，结果中没有key2列。这是因为
df['key2']不是数值数据（俗称“麻烦列”），所以被从结果中排除了。默认情况下，所有数值列都会被聚合，虽然有时可能会被过
滤为⼀个⼦集，稍后就会碰到

⽆论你准备拿groupby做什么，都有可能会⽤到GroupBy的size
⽅法，它可以返回⼀个含有分组⼤⼩的Series：

In [18]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

**注意，任何分组关键词中的缺失值，都会被从结果中除去。**

#### 对分组进行迭代
GroupBy对象⽀持迭代，可以产⽣⼀组⼆元元组（由分组名和数
据块组成）。看下⾯的例⼦：

In [21]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.537970 -1.227607
1    a  two  0.324963 -0.852639
4    a  one  0.620406 -1.349194
b
  key1 key2     data1     data2
2    b  one -1.623929 -0.204281
3    b  two  0.332654 -0.241716


对于多重键的情况，元组的第⼀个元素将会是由键值组成的元组

In [22]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.537970 -1.227607
4    a  one  0.620406 -1.349194
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.324963 -0.852639
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.623929 -0.204281
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.332654 -0.241716


当然，你可以对这些数据⽚段做任何操作。有⼀个你可能会觉得
有⽤的运算：将这些数据⽚段做成⼀个字典

In [24]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  1.537970 -1.227607
 1    a  two  0.324963 -0.852639
 4    a  one  0.620406 -1.349194, 'b':   key1 key2     data1     data2
 2    b  one -1.623929 -0.204281
 3    b  two  0.332654 -0.241716}

In [25]:
pieces['a']

,key1,key2,data1,data2
0,a,one,1.537970,-1.227607
1,a,two,0.324963,-0.852639
4,a,one,0.620406,-1.349194


groupby默认是在axis=0上进⾏分组的，通过设置也可以在其他
任何轴上进⾏分组。拿上⾯例⼦中的df来说，我们可以根据
dtype对列进⾏分组：

In [26]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [27]:
grouped = df.groupby(df.dtypes, axis=1)

可以如下打印分组

In [28]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.537970 -1.227607
1  0.324963 -0.852639
2 -1.623929 -0.204281
3  0.332654 -0.241716
4  0.620406 -1.349194
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选取一列或列的子集

对于由DataFrame产⽣的GroupBy对象，如果⽤⼀个（单个字符
串）或⼀组（字符串数组）列名对其进⾏索引，就能实现选取部
分列进⾏聚合的⽬的。也就是说：

In [30]:
df.groupby('key1')['data1']

In [31]:
df.groupby('key1')[['data2']]  # 注意这里是双层括号进行索引

是一下代码的语法糖：

In [32]:
df['data1'].groupby(df['key1'])

In [33]:
df['data2'].groupby(df['key1'])

尤其对于⼤数据集，很可能只需要对部分列进⾏聚合。例如，在
前⾯那个数据集中，如果只需计算data2列的平均值并以
DataFrame形式得到结果，可以这样写：

In [34]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.288401
     two  -0.852639
b    one  -0.204281
     two  -0.241716

这种索引操作所返回的对象是⼀个已分组的DataFrame（如果传
⼊的是列表或数组）或已分组的Series（如果传⼊的是标量形式
的单个列名）：

In [36]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [37]:
s_grouped.mean()

key1  key2
a     one    -1.288401
      two    -0.852639
b     one    -0.204281
      two    -0.241716
Name: data2, dtype: float64